In [6]:
import os
import re
import pandas as pd
from tqdm import tqdm

# Configuración
DATA_DIR = "../data/01_raw/"
OUTPUT_FILE = "../data/02_processed/clean_texts.csv"
CHUNK_SIZE = 500  # Tamaño de fragmentos de texto

In [7]:
def clean_text(text):
    """Realiza una limpieza básica del texto sin tokenización ni lematización."""
    text = text.lower()  # Convertir a minúsculas
    text = re.sub(r'[^\w\s.,!?;:]', '', text)  # Eliminar caracteres especiales excepto puntuación básica
    text = re.sub(r'\s+', ' ', text).strip()  # Remover espacios múltiples
    return text

In [8]:
def chunk_text(text, chunk_size=CHUNK_SIZE):
    """Divide el texto en fragmentos de tamaño específico respetando las palabras completas."""
    words = text.split()  # Dividir el texto en palabras
    chunks = []
    current_chunk = []
    current_length = 0
    
    for word in words:
        if current_length + len(word) + 1 > chunk_size:  # +1 para el espacio entre palabras
            chunks.append(" ".join(current_chunk))
            current_chunk = []
            current_length = 0
        
        current_chunk.append(word)
        current_length += len(word) + 1  # Considerar el espacio
    
    if current_chunk:  # Agregar el último fragmento si quedó texto
        chunks.append(" ".join(current_chunk))
    
    return chunks

In [9]:
def process_text_files(directory):
    """Carga y preprocesa todos los archivos TXT en un directorio."""
    data = []
    
    for filename in tqdm(os.listdir(directory)):
        if filename.endswith(".txt"):
            file_path = os.path.join(directory, filename)
            with open(file_path, "r", encoding="utf-8") as f:
                raw_text = f.read()
                cleaned_text = clean_text(raw_text)
                chunks = chunk_text(cleaned_text)
                for idx, chunk in enumerate(chunks):
                    data.append({"filename": filename, "chunk_id": idx, "clean_text": chunk})
    
    return pd.DataFrame(data)

In [10]:
# Ejecutar procesamiento
df = process_text_files(DATA_DIR)

  0%|          | 0/6 [00:00<?, ?it/s]

100%|██████████| 6/6 [00:00<00:00, 56.86it/s]


In [11]:
df

,filename,chunk_id,clean_text
0,activacion-tarjeta.txt,0,únicamente puedes activar aquí en la aplicació...
1,como-pago-mi-tarjeta-o-credicompras.txt,0,cómo pago mi tarjeta o credicompras? paga en e...
2,como-pago-mi-tarjeta-o-credicompras.txt,1,ahorros o corriente bancolombia al débito auto...
3,como-pago-mi-tarjeta-o-credicompras.txt,2,bancoomeva banco colpatria banco superior ten ...
4,como-pago-mi-tarjeta-o-credicompras.txt,3,a discapacitados nuestros aliados derechos y o...
...,...,...,...
63,tarjetas-de-credito.txt,23,productos que tenemos disponibles en nuestros ...
64,tarjetas-de-credito.txt,24,tarjeta no se encuentra disponible desde el 20...
65,tarjetas-de-credito.txt,25,mastercard black disfruta el privilegio de ser...
66,tarjetas-de-credito.txt,26,aerolínea viva. esta tarjeta no se encuentra d...


In [13]:
# Guardar en CSV
df.to_csv(OUTPUT_FILE, index=False)
print(f"Datos preprocesados guardados en {OUTPUT_FILE}")

Datos preprocesados guardados en ../data/02_processed/clean_texts.csv
